解压数据集

In [1]:
%cd /home/aistudio/data/data100477
!unzip ./常规赛：PALM眼底彩照中黄斑中央凹定位.zip
!mv -f /home/aistudio/data/data100477/常规赛：PALM眼底彩照中黄斑中央凹定位/* /home/aistudio/data

清除保存的模型及结果文件
- 确认这么做的后果之前，请勿运行该代码。

In [2]:
%cd /home/aistudio/work
!rm -rf savepoint
!rm Fovea_Localization_Results.csv
!mkdir -p savepoint

/home/aistudio/work


导入包，设置随机种子。

In [3]:
import os
import math
import numpy as np
import pandas as pd
from PIL import Image
np.random.seed(0)

读取测试集数据的label

In [4]:
trY=pd.read_excel('../data/Train/Fovea_Location_train.xlsx')
trZero=trY[(trY['Fovea_X']==0)&(trY['Fovea_Y']==0)] # 0值特例样本
trZero_list=list(trZero['imgName'])
# trY=trY.drop(trY[(trY['Fovea_X']==0)&(trY['Fovea_Y']==0)].index) # 去除0值特例样本
trX_list=list(trY['imgName'])
trY=np.array(trY[['Fovea_X','Fovea_Y']],dtype=np.float32)
print(len(trX_list))
print(trY.shape)

800
(800, 2)


生成paddle数据集类
- 对图片的预处理如下：
    - resize至[224,224,3]
    - 转换为np.array
    - 除255.0使得像素分布在0~1之间
    - 改变通道位置Channel Last->Channel First
- 对标签的预处理如下：
    - 将标签坐标除以对应维度的分辨率，得到相对坐标
    - 将相对坐标缩放到新的分辨率224*224下

In [5]:
from paddle.io import Dataset
import paddle.vision.transforms as T

class PALMTrData(Dataset):
    def __init__(self,path,pic_list,y):
        super(PALMTrData,self).__init__()
        self.path=path
        self.y=y
        self.pic_list=pic_list
        print(len(self.pic_list))
    
    def __getitem__(self,idx):
        # X-feature
        pic_name=self.pic_list[idx]
        pic=Image.open(self.path+'/'+pic_name)
        pic_size=pic.size
        pic=pic.resize((224,224),Image.ANTIALIAS)
        pic=np.array(pic,dtype=np.float32)
        pic=pic/255.0
        pic=T.to_tensor(pic)
        # Y-label
        label=self.y[idx].copy()
        label[0]=224.0*label[0]/pic_size[0]
        label[1]=224.0*label[1]/pic_size[1]
        return pic,label

    def __len__(self):
        return len(self.pic_list)

class PALMTsData(Dataset):
    def __init__(self,path):
        super(PALMTsData,self).__init__()
        self.path=path
        self.pic_list=os.listdir(path)
        print(len(self.pic_list))
    
    def __getitem__(self,idx):
        pic_name=self.pic_list[idx]
        pic=Image.open(self.path+'/'+pic_name)
        pic=pic.resize((224,224),Image.ANTIALIAS)
        pic=np.array(pic,dtype=np.float32)
        pic=pic/255.0
        pic=T.to_tensor(pic)
        return pic,0

    def __len__(self):
        return len(self.pic_list)

class PALMZeroData(Dataset):
    def __init__(self,path,pic_list):
        super(PALMZeroData,self).__init__()
        self.path=path
        self.pic_list=pic_list
        print(len(self.pic_list))
    
    def __getitem__(self,idx):
        pic_name=self.pic_list[idx]
        pic=Image.open(self.path+'/'+pic_name)
        pic=pic.resize((224,224),Image.ANTIALIAS)
        pic=np.array(pic,dtype=np.float32)
        pic=pic/255.0
        pic=T.to_tensor(pic)
        return pic,0

    def __len__(self):
        return len(self.pic_list)

tr_path='../data/Train/fundus_image'
trData=PALMTrData(path=tr_path,pic_list=trX_list,y=trY)
ts_path='../data/PALM-Testing400-Images'
tsData=PALMTsData(path=ts_path)
zeroData=PALMZeroData(path=tr_path,pic_list=trZero_list)

800
400
6


构建模型，设置随机种子，设置训练设备为GPU
- 模型使用预训练模型resnet50为主体
- 下游任务分类器结构为2层全连接网络(1000->100->2)，以relu激活，并辅以Dropout=0.2防止过拟合
- 下游任务被设定为回归任务，分别回归x,y坐标

In [6]:
import paddle
paddle.seed(0)
paddle.set_device('gpu')
from paddle import nn
import paddle.nn.functional as F
from paddle.vision.models import resnet50

class PALM(nn.Layer):
    def __init__(self):
        super(PALM,self).__init__()
        self.resnet=resnet50(pretrained=True)
        self.do1=nn.Dropout(p=0.2)
        self.lr1=nn.Linear(in_features=1000, out_features=100)
        self.do2=nn.Dropout(p=0.2)
        self.lr2=nn.Linear(in_features=100, out_features=2)

    def forward(self,x):
        x=self.resnet(x)
        x=F.relu(x)
        x=self.do1(x)
        x=self.lr1(x)
        x=F.relu(x)
        x=self.do2(x)
        x=self.lr2(x)
        return x

全局参数设置
- 集成模型数：10
- 训练、预测批大小：16
- 训练回合：50
- 学习率：固定5e-5

In [7]:
bagging=10
batch_size=16
epochs=50
lr=5e-5

模型训练
- 优化器：adam
- Loss：与比赛的最终评价分数相同，欧氏距离=MSELoss
- 数据异步读取：2个进程
- 训练集：全部训练数据
- 验证集：无

In [8]:
from paddle.optimizer import Adam
from paddle.nn import MSELoss
from paddle.metric import Accuracy

inputs=paddle.static.InputSpec([-1, 3, 224, 224], dtype='float32', name='input')
label=paddle.static.InputSpec([-1, 2], dtype='float32', name='label')
model=[]
for i in range(bagging):
    model.append(paddle.Model(PALM(), inputs, label))
    model[i].prepare(optimizer=Adam(learning_rate=lr,parameters=model[i].parameters()),loss=MSELoss(),metrics=Accuracy())
    print('------------start fit model {}------------'.format(i))
    model[i].fit(train_data=trData,batch_size=batch_size,epochs=epochs,verbose=1,num_workers=2)
    print('-------------end fit model {}-------------'.format(i))

100%|██████████| 151272/151272 [00:03<00:00, 43602.73it/s]


------------start fit model 0------------
The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/50


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:641: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


测试数据生成预测结构
- 集成模型结果逐个汇总

In [9]:
preds=np.zeros((bagging,400,2))
for i in range(bagging):
    pps=model[i].predict(tsData,batch_size=batch_size,num_workers=2)
    preds[i,:,:]=np.squeeze(np.array(pps)).reshape((400,2))

Predict begin...
step 25/25 [==============================] - 1s/step            
Predict samples: 400
Predict begin...
step 25/25 [==============================] - 963ms/step        
Predict samples: 400
Predict begin...
step 25/25 [==============================] - 975ms/step         
Predict samples: 400
Predict begin...
step 25/25 [==============================] - 963ms/step         
Predict samples: 400
Predict begin...
step 25/25 [==============================] - 972ms/step         
Predict samples: 400
Predict begin...
step 25/25 [==============================] - 959ms/step         
Predict samples: 400
Predict begin...
step 25/25 [==============================] - 968ms/step        
Predict samples: 400
Predict begin...
step 25/25 [==============================] - 947ms/step        
Predict samples: 400
Predict begin...
step 25/25 [==============================] - 967ms/step        
Predict samples: 400
Predict begin...
step 25/25 [==============================] - 938ms

合并集成模型结果
- 对(x,y)分别取均值
- 将预测的标签缩放回原分辨率
- 保存成提交文件

In [10]:
pred=np.mean(preds,axis=0)
pred_list=os.listdir(ts_path)
print(pred.shape)
for i,pic_name in enumerate(pred_list):
    pic=Image.open(ts_path+'/'+pic_name)
    pic_size=pic.size
    pred[i,0]=pred[i,0]*pic_size[0]/224.0
    pred[i,1]=pred[i,1]*pic_size[1]/224.0
ans=pd.DataFrame({'FileName':pred_list,'Fovea_X':pred[:,0],'Fovea_Y':pred[:,1]})
ans=ans.sort_values(by='FileName',ignore_index=True)
ans.to_csv('./Fovea_Localization_Results.csv',index=0)

(400, 2)


来看看训练集中特殊的6个(0,0)样本
- 预测效果基本接近(0,0)，而不是像正常图片一样位于图像中央附近

In [11]:
for i in range(bagging):
    pred_zero=model[i].predict(zeroData)
    pred_zero=np.squeeze(np.array(pred_zero))
    for i,pic_name in enumerate(trZero_list):
        pic=Image.open(tr_path+'/'+pic_name)
        pic_size=pic.size
        pred_zero[i,0]=pred_zero[i,0]*pic_size[0]/224.0
        pred_zero[i,1]=pred_zero[i,1]*pic_size[1]/224.0
    print(pred_zero)

Predict begin...
step 6/6 [==============================] - 86ms/step        
Predict samples: 6
[[30.977886 29.560293]
 [30.298336 28.31375 ]
 [39.123344 34.739216]
 [28.119568 25.524931]
 [32.980995 30.997164]
 [26.794113 23.975035]]
Predict begin...
step 6/6 [==============================] - 80ms/step        
Predict samples: 6
[[ 62.43268   61.759304]
 [ 61.888336  60.275875]
 [432.57928  407.9349  ]
 [ 57.427326  54.19331 ]
 [ 93.55544   86.48122 ]
 [ 50.160126  49.298267]]
Predict begin...
step 6/6 [==============================] - 82ms/step        
Predict samples: 6
[[39.2759   34.780766]
 [40.383167 36.43218 ]
 [58.805923 48.587856]
 [33.8789   29.17144 ]
 [48.40486  40.310764]
 [35.168938 32.21965 ]]
Predict begin...
step 6/6 [==============================] - 83ms/step        
Predict samples: 6
[[43.494564 45.781975]
 [48.071712 55.601288]
 [68.41006  74.72025 ]
 [56.565243 61.780388]
 [70.04275  76.33488 ]
 [48.51297  47.58485 ]]
Predict begin...
step 6/6 [=============

模型参数保存

In [14]:
for i in range(bagging):
    model[i].save('./savepoint/final_{}'.format(i),training=False)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:322: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/vision/models/resnet.py:143
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_add(X, Y, axis=0) instead of A + B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
